In [182]:
pip install tableone


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [183]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from tableone import TableOne
from scipy import stats
from pandas.api.types import is_string_dtype, is_numeric_dtype
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [184]:
url = 'https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv'

df = pd.read_csv(url)
df

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,56037,43051,6104,14.178532,6326,14.694200,5359,12.448027,6577,15.277229,...,2098,8.9,8.3,9.6,2834,2.6,2.4,2.8,821,5
3136,56039,23081,2384,10.328842,2185,9.466661,2967,12.854729,4093,17.733200,...,928,7.2,6.5,8.0,1360,2.4,2.2,2.6,447,5
3137,56041,20299,3121,15.375142,3205,15.788955,2153,10.606434,2702,13.311001,...,1163,10.4,9.5,11.2,1500,3.0,2.8,3.2,430,5
3138,56043,7885,858,10.881420,1113,14.115409,715,9.067850,903,11.452124,...,506,11.3,10.3,12.1,686,3.4,3.2,3.7,207,6


In [185]:
for col in df.columns:
    print(df[col].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [186]:
df.describe()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
count,3140.000000,3.140000e+03,3.140000e+03,3140.000000,3.140000e+03,3140.000000,3.140000e+03,3140.000000,3.140000e+03,3140.000000,...,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000
mean,30401.640764,1.041894e+05,1.274030e+04,11.871051,1.336798e+04,12.694609,1.446933e+04,12.283979,1.391649e+04,11.751535,...,5827.242357,13.073503,12.088089,14.053726,9326.577707,3.446242,3.207516,3.710478,2466.234076,4.635350
std,15150.559265,3.335834e+05,4.180730e+04,2.124081,4.228439e+04,1.815044,4.957773e+04,3.126297,4.899095e+04,1.696599,...,15720.551934,2.724351,2.622948,2.824828,29754.601185,0.568059,0.527740,0.613069,7730.422067,1.510447
min,1001.000000,8.800000e+01,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,1.100000e+01,6.092789,...,7.000000,6.100000,5.500000,6.700000,11.000000,1.800000,1.700000,1.900000,3.000000,1.000000
25%,18180.500000,1.096325e+04,1.280500e+03,10.594639,1.374500e+03,11.674504,1.263750e+03,10.496774,1.232750e+03,10.689322,...,815.000000,11.200000,10.300000,12.100000,1187.750000,3.100000,2.900000,3.300000,314.750000,3.000000
50%,29178.000000,2.580050e+04,3.057000e+03,11.802727,3.274000e+03,12.687422,3.108000e+03,11.772649,3.000500e+03,11.580861,...,1963.500000,12.800000,11.800000,13.800000,2743.000000,3.400000,3.200000,3.700000,718.000000,5.000000
75%,45081.500000,6.791300e+04,8.097000e+03,12.951840,8.822250e+03,13.659282,8.976250e+03,13.182260,8.314250e+03,12.639379,...,4727.000000,14.800000,13.700000,15.900000,6679.250000,3.800000,3.500000,4.100000,1776.250000,6.000000
max,56045.000000,1.010552e+07,1.208253e+06,25.460677,1.239139e+06,23.304372,1.557073e+06,37.570198,1.501844e+06,22.225129,...,434075.000000,25.600000,24.200000,27.000000,952335.000000,6.200000,5.800000,6.600000,237766.000000,6.000000


**TABLE ONE**

In [187]:
# funcion para indetificar columnas binarias
def is_bool(df, cols):
  df_ = df.copy()
  bool_vars = []
  for col in cols:
    if (df[col].dtype == int) or (df[col].dtype == float):
      flag = True
      unique_list = df[col].unique()
      for value in unique_list:
        if (value not in [1, 0, 0.0, 1.0]) and (np.isnan(value) == False):
          flag = False
      if flag == True:
        bool_vars.append(col)
  return bool_vars

def breakdown_vars(df):
  categorical_vars = []
  boolean_vars = []
  normal_vars = []
  non_normal_vars = []
  boolean_vars = is_bool(df, df.columns)
  for col in df.columns:
    if (df[col].dtype == 'object' or df[col].dtype == 'category') and col not in boolean_vars:
      categorical_vars.append(col)
    elif (df[col].dtype == 'int64' or df[col].dtype == 'float64') and col not in boolean_vars:
      n, p = stats.shapiro(df[col])
      if p < 0.05:
        non_normal_vars.append(col)
      else:
        normal_vars.append(col)
  return categorical_vars,  boolean_vars, normal_vars, non_normal_vars

In [188]:
categorical_vars, boolean_vars, normal_vars, non_normal_vars =  breakdown_vars(df)
categorical_vars, boolean_vars, normal_vars, non_normal_vars

(['COUNTY_NAME', 'STATE_NAME'],
 [],
 [],
 ['fips',
  'TOT_POP',
  '0-9',
  '0-9 y/o % of total pop',
  '19-Oct',
  '10-19 y/o % of total pop',
  '20-29',
  '20-29 y/o % of total pop',
  '30-39',
  '30-39 y/o % of total pop',
  '40-49',
  '40-49 y/o % of total pop',
  '50-59',
  '50-59 y/o % of total pop',
  '60-69',
  '60-69 y/o % of total pop',
  '70-79',
  '70-79 y/o % of total pop',
  '80+',
  '80+ y/o % of total pop',
  'White-alone pop',
  '% White-alone',
  'Black-alone pop',
  '% Black-alone',
  'Native American/American Indian-alone pop',
  '% NA/AI-alone',
  'Asian-alone pop',
  '% Asian-alone',
  'Hawaiian/Pacific Islander-alone pop',
  '% Hawaiian/PI-alone',
  'Two or more races pop',
  '% Two or more races',
  'POP_ESTIMATE_2018',
  'N_POP_CHG_2018',
  'GQ_ESTIMATES_2018',
  'R_birth_2018',
  'R_death_2018',
  'R_NATURAL_INC_2018',
  'R_INTERNATIONAL_MIG_2018',
  'R_DOMESTIC_MIG_2018',
  'R_NET_MIG_2018',
  'Less than a high school diploma 2014-18',
  'High school diploma 

In [189]:
my_table = TableOne(df,
                    categorical= categorical_vars,
                    nonnormal = non_normal_vars,
                    groupby = 'STATE_NAME',
                    pval = True,
                    htest_name=True
                    )
my_table

Grouped by STATE_NAME                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
                                                                                                                                  Missing                    Overall                    Alabama                     Alaska                      Arizona                   Arkansas                   California                   Colorado                   Connecticut                      Delaware          District of Columbia                      Florida                    Georgia                       Hawaii                      Idaho                   Illinois                    Indiana                       Iowa                     Kansas                   Kentucky                  Louisiana                       Maine                     Maryland                 Massachusetts                   Michigan                  Minnesota                Mississippi                   Missouri                    Montana                   Nebraska                     Nevada               New Hampshire                    New Jersey                 New Mexico                    New York              North Carolina               North Dakota                        Ohio                   Oklahoma                      Oregon                Pennsylvania                 Rhode Island              South Carolina               South Dakota                  Tennessee                      Texas                       Utah                    Vermont                   Virginia                  Washington              West Virginia                  Wisconsin                    Wyoming P-Value                                       Test
n                                                                                                                                                               3140                         67                         27                           15                         75                           58                         64                             8                             3                             1                           67                        159                            5                         44                        102                         92                         99                        105                        120                         64                          16                           24                            14                         83                         87                       

In [190]:
# my_table.to_excel('tableone_output.xlsx')

**ONE HOT ENCODER**

In [191]:
#identificar variables numericas del df filtrado
numerical_vars = [col for col in df.columns if is_numeric_dtype(df[col])]

#identificar variables categoricas del df filtrado
categorical_vars = [col for col in df.columns if is_string_dtype(df[col])]

numerical_vars, categorical_vars

(['fips',
  'TOT_POP',
  '0-9',
  '0-9 y/o % of total pop',
  '19-Oct',
  '10-19 y/o % of total pop',
  '20-29',
  '20-29 y/o % of total pop',
  '30-39',
  '30-39 y/o % of total pop',
  '40-49',
  '40-49 y/o % of total pop',
  '50-59',
  '50-59 y/o % of total pop',
  '60-69',
  '60-69 y/o % of total pop',
  '70-79',
  '70-79 y/o % of total pop',
  '80+',
  '80+ y/o % of total pop',
  'White-alone pop',
  '% White-alone',
  'Black-alone pop',
  '% Black-alone',
  'Native American/American Indian-alone pop',
  '% NA/AI-alone',
  'Asian-alone pop',
  '% Asian-alone',
  'Hawaiian/Pacific Islander-alone pop',
  '% Hawaiian/PI-alone',
  'Two or more races pop',
  '% Two or more races',
  'POP_ESTIMATE_2018',
  'N_POP_CHG_2018',
  'GQ_ESTIMATES_2018',
  'R_birth_2018',
  'R_death_2018',
  'R_NATURAL_INC_2018',
  'R_INTERNATIONAL_MIG_2018',
  'R_DOMESTIC_MIG_2018',
  'R_NET_MIG_2018',
  'Less than a high school diploma 2014-18',
  'High school diploma only 2014-18',
  "Some college or associat

In [192]:
#one hot encoder para variables categoricas
onehot_encoder = OneHotEncoder(handle_unknown='ignore', feature_name_combiner='concat')

#obtener data transformada
transformed_data = onehot_encoder.fit_transform(df[categorical_vars]).toarray()

#obtener nombres de variables
feature_names = onehot_encoder.get_feature_names_out(categorical_vars)

# crear dataframe con data transformada y nombres de variables
ohe_df = pd.DataFrame(transformed_data, columns=feature_names)
ohe_df

,COUNTY_NAME_Abbeville,COUNTY_NAME_Acadia,COUNTY_NAME_Accomack,COUNTY_NAME_Ada,COUNTY_NAME_Adair,COUNTY_NAME_Adams,COUNTY_NAME_Addison,COUNTY_NAME_Aiken,COUNTY_NAME_Aitkin,COUNTY_NAME_Alachua,...,STATE_NAME_South Dakota,STATE_NAME_Tennessee,STATE_NAME_Texas,STATE_NAME_Utah,STATE_NAME_Vermont,STATE_NAME_Virginia,STATE_NAME_Washington,STATE_NAME_West Virginia,STATE_NAME_Wisconsin,STATE_NAME_Wyoming
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [193]:
#escalamiento minmax de variables numericas originales
scaler = MinMaxScaler()

scaled_features = scaler.fit_transform(df[numerical_vars])
df_scaled = pd.DataFrame(scaled_features, index = df.index, columns = numerical_vars)

df_scaled = df_scaled.join(ohe_df)
df_scaled

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,STATE_NAME_South Dakota,STATE_NAME_Tennessee,STATE_NAME_Texas,STATE_NAME_Utah,STATE_NAME_Vermont,STATE_NAME_Virginia,STATE_NAME_Washington,STATE_NAME_West Virginia,STATE_NAME_Wisconsin,STATE_NAME_Wyoming
0,0.000000,0.005493,0.005617,0.479430,0.006163,0.589390,0.004417,0.329258,0.004713,0.412648,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000036,0.021566,0.020490,0.445993,0.021719,0.529693,0.015143,0.287860,0.016781,0.339172,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000073,0.002453,0.002261,0.431264,0.002389,0.510489,0.002099,0.349599,0.002124,0.419806,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000109,0.002208,0.002033,0.430636,0.002095,0.497301,0.001945,0.359921,0.002065,0.483782,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000145,0.005715,0.005872,0.481786,0.006109,0.561604,0.004330,0.310254,0.004576,0.360085,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,0.999855,0.004251,0.005052,0.556880,0.005105,0.630534,0.003442,0.331327,0.004372,0.569319,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3136,0.999891,0.002275,0.001973,0.405678,0.001763,0.406218,0.001905,0.342152,0.002718,0.721558,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3137,0.999927,0.002000,0.002583,0.603878,0.002586,0.677510,0.001383,0.282310,0.001792,0.447437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3138,0.999964,0.000772,0.000710,0.427381,0.000898,0.605698,0.000459,0.241358,0.000594,0.332211,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [194]:
df_corr = df_scaled.corr()

In [195]:
# df_corr.to_excel('correlation_matrix.xlsx')

In [196]:
df_scaled.rename(columns = {'Bachelor\'s degree or higher 2014-18':'Bachelors degree or higher 2014-18', 'Percent of adults with a bachelor\'s degree or higher 2014-18' : 'Percent of adults with a bachelors degree or higher 2014-18' })

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,STATE_NAME_South Dakota,STATE_NAME_Tennessee,STATE_NAME_Texas,STATE_NAME_Utah,STATE_NAME_Vermont,STATE_NAME_Virginia,STATE_NAME_Washington,STATE_NAME_West Virginia,STATE_NAME_Wisconsin,STATE_NAME_Wyoming
0,0.000000,0.005493,0.005617,0.479430,0.006163,0.589390,0.004417,0.329258,0.004713,0.412648,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000036,0.021566,0.020490,0.445993,0.021719,0.529693,0.015143,0.287860,0.016781,0.339172,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000073,0.002453,0.002261,0.431264,0.002389,0.510489,0.002099,0.349599,0.002124,0.419806,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000109,0.002208,0.002033,0.430636,0.002095,0.497301,0.001945,0.359921,0.002065,0.483782,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000145,0.005715,0.005872,0.481786,0.006109,0.561604,0.004330,0.310254,0.004576,0.360085,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,0.999855,0.004251,0.005052,0.556880,0.005105,0.630534,0.003442,0.331327,0.004372,0.569319,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3136,0.999891,0.002275,0.001973,0.405678,0.001763,0.406218,0.001905,0.342152,0.002718,0.721558,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3137,0.999927,0.002000,0.002583,0.603878,0.002586,0.677510,0.001383,0.282310,0.001792,0.447437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3138,0.999964,0.000772,0.000710,0.427381,0.000898,0.605698,0.000459,0.241358,0.000594,0.332211,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [197]:
# #obesity prevalence como variable objetivo
# df_scaled = df_scaled[['Bachelor\'s degree or higher 2014-18',
# 'Active Primary Care Physicians per 100000 Population 2018 (AAMC)',
# '% Asian-alone',
# 'Percent of adults with a bachelor\'s degree or higher 2014-18',
# 'Percent of adults with less than a high school diploma 2014-18',
# '% Black-alone',
# 'Obesity_prevalence']]

# # 'Percent of adults with a high school diploma only 2014-18', 'R_death_2018', 'PCTPOVALL_2018', 'Unemployment_rate_2018',

# df_scaled

In [198]:
# # VIF dataframe
# vif_data = pd.DataFrame()
# vif_data["feature"] = df_scaled.columns

# # calculating VIF for each feature
# vif_data["VIF"] = [variance_inflation_factor(df_scaled.values, i)
#                           for i in range(len(df_scaled.columns))]

# vif_data

In [199]:
#division en train y test
X = df_scaled.drop('Obesity_prevalence', axis=1)
y = df_scaled['Obesity_prevalence']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size  = 0.3, random_state=123)
X_train

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,STATE_NAME_South Dakota,STATE_NAME_Tennessee,STATE_NAME_Texas,STATE_NAME_Utah,STATE_NAME_Vermont,STATE_NAME_Virginia,STATE_NAME_Washington,STATE_NAME_West Virginia,STATE_NAME_Wisconsin,STATE_NAME_Wyoming
1272,0.455744,0.008702,0.007497,0.404149,0.008968,0.541718,0.006470,0.304636,0.006272,0.286434,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
693,0.294310,0.028103,0.029943,0.500201,0.029776,0.557314,0.022856,0.333450,0.022873,0.372135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1617,0.527614,0.001188,0.000995,0.390263,0.001227,0.539173,0.000640,0.219368,0.000824,0.262335,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1711,0.547162,0.003494,0.004281,0.573962,0.003904,0.586453,0.002941,0.344367,0.003009,0.415731,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193,0.091163,0.098394,0.130665,0.623570,0.122083,0.652796,0.097093,0.404662,0.092638,0.489660,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154,0.709432,0.005488,0.005692,0.486225,0.006307,0.603670,0.004099,0.305789,0.004766,0.422175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3089,0.982596,0.018532,0.019730,0.499723,0.019940,0.565864,0.015290,0.338214,0.017111,0.472874,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1766,0.581498,0.008876,0.006255,0.330619,0.008932,0.528960,0.008555,0.394898,0.006721,0.319963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1122,0.381913,0.000681,0.000662,0.450933,0.000716,0.546233,0.000575,0.342260,0.000563,0.383822,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**REGRESION LINEAL**

In [200]:
linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train, y_train)

LinearRegression()

In [201]:
linear_regression_model.coef_

array([-0.06903722,  2.57478349,  0.08922996, ...,  0.08997689,
        0.28264806, -0.0882387 ], shape=(1997,))

In [202]:
y_pred = linear_regression_model.predict(X_test)
y_pred

array([ 0.44443759,  0.56908262,  0.6084997 ,  0.25558022,  0.77320797,
        0.50366751,  0.3487763 ,  0.85135345,  0.58309856,  0.79447366,
        0.52667659,  0.60350994,  0.704364  ,  0.5402407 ,  0.6847606 ,
        0.72269075,  0.60039047,  0.63485172,  0.56374237,  0.61612418,
        0.42804962,  0.36239321,  0.44357887,  0.53710665,  0.64665283,
        0.48398859,  0.63537051,  0.44787983,  0.87478627,  0.6771161 ,
        0.70868629,  0.68665929,  0.63061716,  0.40856325,  0.6351001 ,
        0.59836029,  0.66309773,  0.62100398,  0.59626059,  0.62291808,
        0.66826947,  0.575601  ,  0.81213803,  0.46621369,  0.58067393,
       -1.26323489,  0.71017428,  0.53705487,  0.69300317,  0.88302212,
        0.76110477,  0.62097941,  0.21997924,  0.58525758,  0.22018847,
        0.70556651,  0.58186402,  1.09979622,  0.60684879,  0.64541829,
        0.73049153,  0.64222973,  0.6019138 ,  0.57022458,  0.45375352,
        0.69780631,  0.85368308,  0.54020578,  0.5193094 ,  0.71

In [203]:
r2 = linear_regression_model.score(X_test, y_test)
r2

-0.4948071913736749

In [204]:
mean_squared_error(y_test, y_pred)

0.025716962510871962

**REGRESION LASSO**

In [205]:
lasso_model = Lasso(alpha = 10)
lasso_model.fit(X_train, y_train)
lasso_model

Lasso(alpha=10)

In [206]:
y_pred = lasso_model.predict(X_test)
y_pred

array([0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011089,
       0.57011089, 0.57011089, 0.57011089, 0.57011089, 0.57011

In [207]:
r2 = lasso_model.score(X_test, y_test)
r2

-0.0007126734056219775

In [208]:
mean_squared_error(y_test, y_pred)

0.01721646139692238

In [209]:
def lasso_gridcv(X_train, y_train):
    model =  Lasso()
    hyperparams = {
        "alpha" : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
        "fit_intercept" :  [True, False],
        "max_iter" : [15000, 30000, 60000]
        }
    cv = kfold = KFold(n_splits = 10, shuffle=True, random_state=123)
    grid_search = GridSearchCV(estimator=model,
                               param_grid=hyperparams,
                               cv=cv,
                               scoring= 'neg_mean_squared_error',)
    grid_result = grid_search.fit(X_train, y_train)
    return grid_result.best_estimator_

In [210]:
best_lasso = lasso_gridcv(X_train, y_train)
best_lasso

Lasso(alpha=0.0001, max_iter=15000)

In [211]:
y_pred = best_lasso.predict(X_test)
y_pred

array([0.4418382 , 0.56845446, 0.60531868, 0.25922894, 0.77516452,
       0.50355424, 0.34612523, 0.84665555, 0.57971572, 0.79322865,
       0.52490809, 0.60287052, 0.70059676, 0.53786627, 0.68168953,
       0.55231345, 0.59958242, 0.63431407, 0.55984888, 0.6184883 ,
       0.42802432, 0.36036228, 0.43928756, 0.53617094, 0.64250088,
       0.48570028, 0.63420761, 0.44927226, 0.86960066, 0.67415465,
       0.70473964, 0.68254578, 0.62751308, 0.40593379, 0.63463043,
       0.59374101, 0.66320639, 0.6167897 , 0.59639955, 0.62390858,
       0.57727011, 0.57504739, 0.80289958, 0.4631919 , 0.57802009,
       0.23682498, 0.70867515, 0.53372497, 0.69146776, 0.88182328,
       0.76050866, 0.62156773, 0.2227978 , 0.58503302, 0.67128459,
       0.69868767, 0.57833317, 0.63538068, 0.60649144, 0.64132843,
       0.72843504, 0.64153544, 0.60106834, 0.56654737, 0.45320685,
       0.69561102, 0.84983877, 0.53935439, 0.51565609, 0.71579208,
       0.62401335, 0.58970332, 0.53520793, 0.52363165, 0.49654

In [212]:
mean_squared_error(y_test, y_pred)

3.2895924277229273e-06

In [213]:
r2 = best_lasso.score(X_test, y_test)
r2

0.9998087913214646